In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as tfs
import warnings

warnings.filterwarnings('ignore')

In [2]:
from MyKu import processing

train_data = processing.get_exist2021_data(type='train', len=3437)
test_data = processing.get_exist2021_data(type='test', len=1000)

In [3]:

model_class, tokenizer_class, pretrained_weights = (
    tfs.BertModel, tfs.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

train_tokenized = [tokenizer.encode(
    x, add_special_tokens=True) for x in train_data[0]]

train_max_len = 0
for i in train_tokenized:
    if len(i) > train_max_len:
        train_max_len = len(i)

train_padded = np.array([i + [0] * (train_max_len-len(i))
                        for i in train_tokenized])
print("train set shape:", train_padded.shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


train set shape: (3437, 80)


In [4]:
train_attention_mask = np.where(train_padded != 0, 1, 0)

train_input_ids = torch.tensor(train_padded).long()
train_attention_mask = torch.tensor(train_attention_mask).long()
with torch.no_grad():
    train_last_hidden_states = model(
        train_input_ids, attention_mask=train_attention_mask)

print(train_last_hidden_states[0].size())

torch.Size([3437, 80, 768])


In [ ]:
train_features = train_last_hidden_states[0][:, 0, :].numpy()
train_labels = train_data[1]
train_features, test_features, train_labels, test_labels = train_test_split(
    train_features, train_labels)

In [7]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
ss = lr_clf.score(test_features, test_labels)
print(ss)

0.713953488372093
